In [61]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-ml-w5p2/trainX.csv
/kaggle/input/2024-ml-w5p2/testX.csv
/kaggle/input/2024-ml-w5p2/trainY.csv
/kaggle/input/2024-ml-w5p2/submit.csv


In [62]:
# 랜덤시드 고정
import random
seed = 42

random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [63]:
# 데이터셋 로드
trainX = pd.read_csv('/kaggle/input/2024-ml-w5p2/trainX.csv')
trainY = pd.read_csv('/kaggle/input/2024-ml-w5p2/trainY.csv')
testX  = pd.read_csv('/kaggle/input/2024-ml-w5p2/testX.csv')
submit = pd.read_csv('/kaggle/input/2024-ml-w5p2/submit.csv')

In [64]:
# trainX 확인
print(trainX.shape)
trainX.head()

(23, 14)


,sleep_time_mean,sleep_time_std,sleep_time_min,sleep_time_25,sleep_time_50,sleep_time_75,sleep_time_max,sleep_quality_mean,sleep_quality_std,sleep_quality_min,sleep_quality_25,sleep_quality_50,sleep_quality_75,sleep_quality_max
0,6.250000,0.689202,5.0,6.125,6.5,6.50,7.0,3.500000,0.836660,2.0,3.25,4.0,4.0,4.0
1,6.418182,1.169693,3.0,6.000,6.5,6.75,9.5,3.345455,0.672700,1.0,3.00,3.0,4.0,4.0
2,7.034483,1.127262,5.5,6.500,6.5,8.00,11.0,3.862069,0.347839,3.0,4.00,4.0,4.0,4.0
3,6.453846,1.070642,3.0,6.000,6.5,7.00,9.0,3.061538,0.768052,1.0,3.00,3.0,4.0,4.0
4,5.902439,1.817345,3.5,4.500,5.5,7.00,10.0,2.975610,0.961452,1.0,2.00,3.0,4.0,4.0


In [65]:
#trainY 확인
print(trainY.shape)
trainY.head()

(23, 1)


,label
0,0
1,0
2,0
3,1
4,0


In [66]:
trainY.label.unique() #label 값 확인

array([0, 1])

In [67]:
#라벨 데이터셋 저장
y = trainY['label'].values.ravel()

In [68]:
#데이터 전처리
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(trainX, y, test_size=0.2, random_state=seed, stratify=y)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(18, 14) (18,)
(5, 14) (5,)


In [69]:
#데이터 정규화
from sklearn.preprocessing import StandardScaler
std = StandardScaler()

x_train_std = std.fit_transform(x_train)
x_test_std = std.fit_transform(x_test)

In [70]:
#로지스틱 회귀
from sklearn.linear_model import LogisticRegression

Logit = LogisticRegression(class_weight='balanced', random_state=seed)
Logit.fit(x_train_std, y_train)

LogisticRegression(class_weight='balanced', random_state=42)

In [71]:
#예측
y_test_pred = Logit.predict(x_test_std)

#정확도 출력
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_test_pred))

#confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_test_pred))

0.8
[[3 1]
 [0 1]]


In [72]:
#테스트 데이터 예측
trainX_std = std.fit_transform(trainX)
testX_std = std.fit_transform(testX)

Logit.fit(trainX, y)

test_pred = Logit.predict(testX)
test_pred

array([1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0])

In [73]:
#제출 포맷 확인
submit.head()

,id,label
0,1,1.0
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN


In [74]:
#submit 제출
submit['label'] = test_pred
submit.to_csv('submit.csv', index=False)